# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [4]:
!pip3 install pymongo

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [5]:
# Import dependencies
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [6]:
pymongo.__version__

'4.10.1'

In [7]:
username = 'ShannonA'
password = 'Shannon01'
host = "localhost"
port = 27017
database = "cluster0"


In [8]:
client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}/{database}')


In [ ]:
# confirm that our new database was created
databases = client.list_database_names()

In [53]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [55]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [57]:
# review the collections in our new database

In [59]:
# review a document in the establishments collection

In [61]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [65]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 Greenwich High Road",
    "AddressLine2": "Greenwich",
    "PostCode": "SE10 8JL",
    "LocalAuthorityName": "Greenwich",
    "scores": {
        "Hygiene": None,
        "Structural": None,
        "ConfidenceInManagement": None
    },
    "geocode": {
        "latitude": "51.482",
        "longitude": "-0.009"
    },
    "RatingValue": "Awaiting Inspection"
}


In [87]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67c139c2ffa655eac6f3ddfb, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# Check that the new restaurant was inserted
if result.acknowledged:
    print(f"Document inserted with _id: {result.inserted_id}")
else:
    print("Document insertion failed.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(business_type)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
)

In [ ]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover:", dover_count)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

In [ ]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover after deletion:", dover_count_after)

In [ ]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {"$set": {"geocode.latitude": {"$toDecimal": "$geocode.latitude"}}},
        {"$set": {"geocode.longitude": {"$toDecimal": "$geocode.longitude"}}}
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many(
    {"RatingValue": {"$in": non_ratings}},
    {"$set": {"RatingValue": None}}
)

In [ ]:
# Check that the coordinates and rating value are now numbers
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

pprint(establishments.find_one({}, {"geocode.latitude": 1, "geocode.longitude": 1, "RatingValue": 1, "_id": 0}))
